### Helpers

In [1]:
import asyncio
from playwright.async_api import async_playwright
from bs4 import BeautifulSoup, Comment
import os
import tiktoken
import json
import pandas as pd
from src.firefall import FirefallService
from src.imss import IMSService
from src.utils import Utils as utils

adobe_env = 'prod'
ims_service = IMSService(adobe_env)
firefall_service = FirefallService(ims_service.ACCESS_TOKEN, 'prod')

async def wait_for_page_load(page, timeout=30000):
    """
    Waits for a page to load by checking if the page size has changed.
    If the page size has not changed within the specified timeout, it assumes that the page has finished loading.

    Args:
        page: The page object to wait for.
        timeout: The maximum time to wait for the page to load, in milliseconds. Defaults to 30000 (30 seconds).

    Raises:
        asyncio.TimeoutError: If the page does not load within the specified timeout.
    """
    # Convert timeout from milliseconds to seconds
    timeout_in_seconds = timeout / 1000

    # Get the initial page size
    initial_page_size = len(await page.content())
    print('Waiting for page to load...')
    try:
        await asyncio.wait_for(_wait_for_page_load(page, initial_page_size), timeout_in_seconds)
    except asyncio.TimeoutError:
        print('Timeout waiting for page to load')

async def _wait_for_page_load(page, initial_page_size):
    """
    Helper function that waits for a page to load by checking if the page size has changed.

    Args:
        page: The page object to wait for.
        initial_page_size: The initial size of the page.

    Returns:
        None
    """
    while True:
        try: 
            await page.wait_for_load_state('networkidle', timeout=5000)
        except Exception:
            pass

        # Get the current page size
        current_page_size = len(await page.content())

        # If the page size has not changed, break the loop
        if current_page_size == initial_page_size:
            break

        # Update the initial page size
        initial_page_size = current_page_size

async def _is_url_streaming_service(url):
    """
    Helper function that checks if a URL is a streaming service.

    Args:
        url: The URL to check.

    Returns:
        bool: True if the URL is a streaming service, False otherwise.
    """
    # List of streaming services
    streaming_services = [
        'youtube.com',
        'vimeo.com',
        'tv.adobe.com'
    ]

    # Check if the URL is a streaming service
    for service in streaming_services:
        if service in url:
            return True
    return False

async def scrape_html(url):
    """
    Scrapes the HTML content from a given URL.

    Args:
        url (str): The URL to scrape.

    Returns:
        str: The HTML content of the page.
    """
    try:
        async with async_playwright() as p:
            # Open the browser and create a new page
            # TODO: Some sites don't work with headless browsers. How can we handle this?
            browser = await p.chromium.launch(headless=True)
            context = await browser.new_context(            )
            #http error
            page = await context.new_page()

            await page.goto(url)
            await wait_for_page_load(page, timeout=30000)

            # Does the page have any visible iframes?
            frames = page.frames
            visible_frame = None
            for frame in frames:
                # The frame is the top-level frame. Skip it.
                if frame.parent_frame is None:
                    continue

                # Is it a nested frame?
                # TODO: How should we handle nested frames?
                if frame.parent_frame.parent_frame is not None:
                    continue

                # Is the frame visible?
                # TODO: Handle more than one visible frame
                frame_element = await frame.frame_element()
                if await frame_element.is_visible():
                    print('Found visible iframe')
                    visible_frame = frame

            # If there is a visible iframe, extract the HTML content from it
            if visible_frame is not None:
                content = await visible_frame.content()
            else:
                content = await page.content()
            #content = await page.content()
            
            return content
    except asyncio.TimeoutError as e:
        # TODO: Log the url that failed to load and continue
        print(e)
        pass
    except Exception as e:
        print(e)
        # Unknown error. Log the error and fail the test
        raise e

async def parse_html(html):
    # Parse the HTML
    soup = BeautifulSoup(html, 'html.parser')

    # Strip all script and style elements
    for script in soup(["script", "style"]):
        script.extract()

    # Strip all link elements
    for link in soup(["link"]):
        link.extract()

    # Strip all HTML comments
    for element in soup(text=lambda text: isinstance(text, Comment)):
        element.extract()

    # Strip all img or svg elements
    for element in soup(["img", "svg"]):
        element.extract()

    # Strip all meta elements
    for element in soup(["meta"]):
        element.extract()

    # Minify the HTML
    soup = soup.prettify(formatter='minimal')

    return str(soup)

def html_to_text(html):
    soup = BeautifulSoup(html, 'html.parser')
    text = soup.get_text()
    text = text.strip()
    text = os.linesep.join([s for s in text.splitlines() if s])
    text = os.linesep.join([s.strip() for s in text.splitlines()])
    text = os.linesep.join([s for s in text.splitlines() if s])
    text = text.replace(u'\xa0', u' ')
    return text


<Response [200]>


### Read Top Pages JSON and convert it into dataframe

In [5]:
#Change the name of the webpage
name_of_webpage = 'lovesac'

#Update top_pages_json
top_pages_json = '/Users/ndere/Downloads/www.lovesac.com-Top_pages-top-keyword-1718024254526 (2).json'
with open(top_pages_json) as f:
    data = json.load(f)

df = pd.DataFrame(data['pages'])
df['file_name_with_extension'] = df['url'].apply(lambda x: x.split('/')[-1] + '.md')

df_path = f'/Users/ndere/Documents/aem-genai/scrape-content/data/{name_of_webpage}/{name_of_webpage}_top200.csv'
df.to_csv(df_path, index=False)

### Scrape content from csv file and add keyword: Keyword at the top of the scraped content, real URLs from the CSV

In [6]:
for i in range(0, len(df)):
    print(f'Processing row {i}')
    url = df.iloc[i]['url']
    file_name_with_extension = df.iloc[i]['file_name_with_extension']
    print(url)

    try:
        html = await parse_html(await scrape_html(url))

        text = html_to_text(html)
        enc = tiktoken.encoding_for_model("gpt-4")

        session_id = 'scraping-20240422'
        prompt = f"""
        Your task is to remove HTML tags and keep only the text content of the page.
        Do not change the text content AT ALL. 
        Remove any unnecessary whitespace and ensure that the text is formatted correctly.
        Remove breadcrumb links, navigation links, and any other non-content text.
        Remove footer text and any other non-content text.
        ```webpage
        {text}
        ```
        """
        response = firefall_service.completions(123, prompt, temperature=0.1, model_name="gpt-4-turbo")
        webpage_outline = response["generations"][0][0]["text"]
        webpage_text = f'Top Keyword: {df.iloc[i]["top_keyword"]}\n{webpage_outline}'
        print(webpage_text)
        
        file_name = f'data/{name_of_webpage}/scraped_content/{file_name_with_extension}'
        utils.read_write_file(file_name,'w', webpage_text)

    except Exception as e:
        print(url + ' failed')


Processing row 0
https://www.lovesac.com/


Future exception was never retrieved
future: <Future finished exception=TargetClosedError('Target page, context or browser has been closed')>
playwright._impl._errors.TargetClosedError: Target page, context or browser has been closed
Future exception was never retrieved
future: <Future finished exception=TargetClosedError('Target page, context or browser has been closed')>
playwright._impl._errors.TargetClosedError: Target page, context or browser has been closed


CancelledError: 

### Scrape content from URL_List array 

In [2]:
#Change the name of the webpage
name_of_webpage = 'lovesac'

url_list = ['https://www.lovesac.com/',
 'https://www.lovesac.com/accessories',
 'https://www.lovesac.com/inspiration',
 'https://www.lovesac.com/learn-about-sactionals',
 'https://www.lovesac.com/learn-about-sactionals-stealthtech-sound-charge',
 'https://www.lovesac.com/sactionals',
 'https://www.lovesac.com/clearance',
 'https://www.lovesac.com/sacs',
 'https://www.lovesac.com/sacs-learn',
 'https://www.lovesac.com/sactionals-with-stealthtech-sound-charge',
 'https://www.lovesac.com/stealthtech-sound-charge-systems',
 'https://www.lovesac.com/about-us',
 'https://www.lovesac.com/accessibility-statement',
 'https://www.lovesac.com/add-on-and-refresh',
 'https://www.lovesac.com/cart',
 'https://www.lovesac.com/checkout',
 'https://www.lovesac.com/contact-us',
 'https://www.lovesac.com/customer-gallery',
 'https://www.lovesac.com/enhance-and-refresh',
 'https://www.lovesac.com/fabric-swatch-guide',
 'https://www.lovesac.com/financing',
 'https://www.lovesac.com/financing-options',
 'https://www.lovesac.com/help-center',
 'https://www.lovesac.com/heroes',
 'https://www.lovesac.com/how-to',
 'https://www.lovesac.com/in-the-news',
 'https://www.lovesac.com/know-your-fabrics',
 'https://www.lovesac.com/lovesac-careers',
 'https://www.lovesac.com/mailing-list',
 'https://www.lovesac.com/online-catalog',
 'https://www.lovesac.com/privacy-policy',
 'https://www.lovesac.com/product-reviews',
 'https://www.lovesac.com/refer-a-friend',
 'https://www.lovesac.com/sacs-inspiration-page',
 'https://www.lovesac.com/ugc-terms',
 'https://www.lovesac.com/stealthtech-setup',
 'https://www.lovesac.com/terms-and-conditions',
 'https://www.lovesac.com/throw-pillows',
 'https://www.lovesac.com/to-the-trade-details',
 'https://www.lovesac.com/sactionals-configuration-guides',
 'https://www.lovesac.com/search',
 'https://www.lovesac.com/squattomans',
 'https://www.lovesac.com/to-the-trade',
 'https://www.lovesac.com/video-gallery',
 'https://www.lovesac.com/throw-pillows/build',
 'https://www.lovesac.com/throw-pillows/covers',
 'https://www.lovesac.com/squattomans/build',
 'https://www.lovesac.com/throw-pillows/covers/select',
 'https://www.lovesac.com/squattomans/covers',
 'https://www.lovesac.com/squattomans/covers/select',
 'https://www.lovesac.com/accessories/blankets-footsacs',
 'https://www.lovesac.com/sacs/accessories',
 'https://www.lovesac.com/sacs/best-sellers',
 'https://www.lovesac.com/sacs/build',
 'https://www.lovesac.com/sacs/covers',
 'https://www.lovesac.com/sacs/inserts',
 'https://www.lovesac.com/sacs/sac-bundles',
 'https://www.lovesac.com/sacs/covers/select',
 'https://www.lovesac.com/sactionals/accessories',
 'https://www.lovesac.com/sactionals/build',
 'https://www.lovesac.com/sactionals/covers',
 'https://www.lovesac.com/sactionals/inserts',
 'https://www.lovesac.com/sactionals/inserts/select',
 'https://www.lovesac.com/sactionals/covers/build',
 'https://www.lovesac.com/sactionals/covers/select',
 'https://www.lovesac.com/stealthtech-firmware-updates',
 'https://www.lovesac.com/pillowsac-accent-chair',
 'https://www.lovesac.com/products/sactionals-drink-holder-dark-walnut',
 'https://www.lovesac.com/products/sactionals-table-dark-walnut',
 'https://www.lovesac.com/products/footsac-blanket-wombat-phur',
 'https://www.lovesac.com/products/footsac-blanket-sodalite-phur',
 'https://www.lovesac.com/products/sactionals-drink-holder-hickory',
 'https://www.lovesac.com/products/sactionals-roll-arm-drink-holder-hickory',
 'https://www.lovesac.com/products/sactionals-roll-arm-drink-holder-dark-walnut',
 'https://www.lovesac.com/products/sactionals-coaster-dark-walnut',
 'https://www.lovesac.com/products/sactionals-coaster-hickory',
 'https://www.lovesac.com/products/sactionals-table-hickory',
 'https://www.lovesac.com/products/footsac-blanket-snow-owl-phur',
 'https://www.lovesac.com/products/footsac-blanket-bronze-wombat-phur',
 'https://www.lovesac.com/products/footsac-blanket-charcoal-wombat-phur',
 'https://www.lovesac.com/products/sactionals-guest-rest-bedding-kit',
 'https://www.lovesac.com/products/footsac-blanket-midnight-mink-phur',
 'https://www.lovesac.com/products/footsac-blanket-glacier-mink-phur',
 'https://www.lovesac.com/products/room-for-two-footsac-blanket-wombat-phur',
 'https://www.lovesac.com/products/room-for-two-footsac-blanket-charcoal-wombat-phur',
 'https://www.lovesac.com/products/room-for-two-footsac-blanket-bronze-wombat-phur',
 'https://www.lovesac.com/products/room-for-two-footsac-blanket-alpine-swirl-phur',
 'https://www.lovesac.com/products/footsac-blanket-alpine-swirl-phur',
 'https://www.lovesac.com/products/radiant-footsac-blanket-wombat-phur',
 'https://www.lovesac.com/products/radiant-footsac-blanket-charcoal-wombat-phur',
 'https://www.lovesac.com/products/radiant-footsac-blanket-alpine-swirl-phur',
 'https://www.lovesac.com/products/footsac-blanket-dove-channeled-phur',
 'https://www.lovesac.com/products/room-for-two-footsac-blanket-dove-channeled-phur',
 'https://www.lovesac.com/products/sactionals-drink-holder-weathered-ash',
 'https://www.lovesac.com/products/sactionals-roll-arm-drink-holder-weathered-ash',
 'https://www.lovesac.com/products/sactionals-coaster-weathered-ash',
 'https://www.lovesac.com/products/sactionals-table-weathered-ash',
 'https://www.lovesac.com/products/room-for-two-footsac-blanket-sterling-hare-phur',
 'https://www.lovesac.com/products/18x18-chinchilla-dense-phur',
 'https://www.lovesac.com/products/24x16-throw-pillow-insert-cover',
 'https://www.lovesac.com/products/24x24-throw-pillow-insert-cover',
 'https://www.lovesac.com/products/room-for-two-footsac-blanket-glacier-mink-phur',
 'https://www.lovesac.com/products/room-for-two-footsac-blanket-midnight-mink-phur',
 'https://www.lovesac.com/products/25th-anniversary-extreme-brilliance-moviesac-bundle-squattoman',
 'https://www.lovesac.com/products/sactionals-angled-side-drink-holder-dark-walnut',
 'https://www.lovesac.com/products/sactionals-angled-side-drink-holder-weathered-ash',
 'https://www.lovesac.com/products/sactionals-angled-side-coaster-dark-walnut',
 'https://www.lovesac.com/products/sactionals-angled-side-coaster-weathered-ash',
 'https://www.lovesac.com/products/footsac-blanket-dusty-blue-long-phur',
 'https://www.lovesac.com/products/room-for-two-footsac-blanket-dusty-blue-long-phur',
 'https://www.lovesac.com/products/room-for-two-footsac-blanket-cloud-scaled-shearling-phur',
 'https://www.lovesac.com/products/footsac-blanket-cloud-scaled-shearling-phur',
 'https://www.lovesac.com/products/footsac-blanket-alabaster-sea-glass-phur',
 'https://www.lovesac.com/products/footsac-blanket-ocean-teal-chenille',
 'https://www.lovesac.com/products/footsac-blanket-khaki-pebbled-phur',
 'https://www.lovesac.com/products/room-for-two-footsac-blanket-khaki-pebbled-phur',
 'https://www.lovesac.com/products/18x18-throw-pillow-cover-alabaster-sea-glass-phur',
 'https://www.lovesac.com/products/18x18-throw-pillow-cover-sage-linen',
 'https://www.lovesac.com/products/18x18-throw-pillow-cover-diamond-stitch',
 'https://www.lovesac.com/products/24x16-throw-pillow-cover-horizontal-fringe-stripe',
 'https://www.lovesac.com/products/24x24-throw-pillow-cover-vertical-wide-stripe',
 'https://www.lovesac.com/products/18x18-throw-pillow-cover-slub-boucle-textured-stripe',
 'https://www.lovesac.com/products/bigone-bundle-squattoman-alabaster-sea-glass-phur',
 'https://www.lovesac.com/products/supersac-bundle-squattoman-alabaster-sea-glass-phur',
 'https://www.lovesac.com/products/moviesac-bundle-squattoman-footsac-alabaster-sea-glass-phur',
 'https://www.lovesac.com/products/moviesac-bundle-squattoman-alabaster-sea-glass-phur',
 'https://www.lovesac.com/products/moviesac-bundle-footsac-alabaster-sea-glass-phur',
 'https://www.lovesac.com/products/citysac-bundle-squattoman-footsac-alabaster-sea-glass-phur',
 'https://www.lovesac.com/products/bigone-bundle-squattoman-ocean-teal-chenille',
 'https://www.lovesac.com/products/citysac-bundle-squattoman-alabaster-sea-glass-phur',
 'https://www.lovesac.com/products/citysac-bundle-footsac-alabaster-sea-glass-phur',
 'https://www.lovesac.com/products/supersac-bundle-squattoman-ocean-teal-chenille',
 'https://www.lovesac.com/products/supersac-bundle-squattoman-footsac-khaki-pebbled-phur',
 'https://www.lovesac.com/products/bigone-bundle-squattoman-khaki-pebbled-phur',
 'https://www.lovesac.com/products/bigone-bundle-footsac-khaki-pebbled-phur',
 'https://www.lovesac.com/products/citysac-bundle-squattoman-footsac-ocean-teal-chenille',
 'https://www.lovesac.com/products/citysac-bundle-squattoman-ocean-teal-chenille',
 'https://www.lovesac.com/products/bigone-bundle-squattoman-footsac-khaki-pebbled-phur',
 'https://www.lovesac.com/products/citysac-bundle-footsac-ocean-teal-chenille',
 'https://www.lovesac.com/products/moviesac-bundle-squattoman-footsac-khaki-pebbled-phur',
 'https://www.lovesac.com/products/supersac-bundle-squattoman-khaki-pebbled-phur',
 'https://www.lovesac.com/products/supersac-bundle-footsac-khaki-pebbled-phur',
 'https://www.lovesac.com/products/moviesac-bundle-squattoman-khaki-pebbled-phur',
 'https://www.lovesac.com/products/moviesac-bundle-footsac-khaki-pebbled-phur',
 'https://www.lovesac.com/products/citysac-bundle-squattoman-footsac-khaki-pebbled-phur',
 'https://www.lovesac.com/products/citysac-bundle-footsac-khaki-pebbled-phur',
 'https://www.lovesac.com/products/citysac-bundle-squattoman-khaki-pebbled-phur',
 'https://www.lovesac.com/products/bigone-bundle-squattoman-alpine-footsac-sodalite-phur',
 'https://www.lovesac.com/products/bigone-bundle-alpine-footsac-sodalite-phur',
 'https://www.lovesac.com/products/supersac-bundle-squattoman-alpine-footsac-sodalite-phur',
 'https://www.lovesac.com/products/supersac-bundle-alpine-footsac-sodalite-phur',
 'https://www.lovesac.com/products/moviesac-bundle-squattoman-alpine-footsac-sodalite-phur',
 'https://www.lovesac.com/products/moviesac-bundle-alpine-footsac-sodalite-phur',
 'https://www.lovesac.com/products/pillowsac-bundle-squattoman-alpine-footsac-sodalite-phur',
 'https://www.lovesac.com/products/pillowsac-bundle-alpine-footsac-sodalite-phur',
 'https://www.lovesac.com/products/citysac-bundle-squattoman-alpine-footsac-sodalite-phur',
 'https://www.lovesac.com/products/citysac-bundle-alpine-footsac-sodalite-phur',
 'https://www.lovesac.com/products/pillowsac-accent-chair-charcoal-wombat-phur-black-webbing-gunmetal-hardware',
 'https://www.lovesac.com/products/pillowsac-accent-chair-onyx-boucle-black-webbing-gunmetal-hardware',
 'https://www.lovesac.com/products/pillowsac-accent-chair-wombat-phur-tan-webbing-nickel-hardware',
 'https://www.lovesac.com/products/pillowsac-accent-chair-dove-channeled-phur-tan-webbing-brass-hardware',
 'https://www.lovesac.com/products/pillowsac-accent-chair-shadow-rained-chenille-black-webbing-gunmetal-hardware',
 'https://www.lovesac.com/products/footsac-blanket-dove-boucle',
 'https://www.lovesac.com/products/footsac-blanket-taupe-waffle-weave',
 'https://www.lovesac.com/products/footsac-blanket-desert-rose-basket-weave']

In [8]:
if not os.path.exists(f'data/{name_of_webpage}/scraped_content'):
    os.makedirs(f'data/{name_of_webpage}/scraped_content')

for i in range(len(url_list)):
    print(f'Processing row {i}')
    url = url_list[i]
    print(url)

    try:
        html = await parse_html(await scrape_html(url))
        text = html_to_text(html)
        enc = tiktoken.encoding_for_model("gpt-4")

        session_id = 'scraping-20240422'
        prompt = f"""
        Your task is to remove HTML tags and keep only the text content of the page. 
        Remove any unnecessary whitespace and ensure that the text is formatted correctly.
        Remove breadcrumb links, navigation links, and any other non-content text.
        Remove footer text and any other non-content text.
        ```webpage
        {text}
        ```

        Provide your response as JSON with the following fields: main_focus (in the webpage's language), sections (numbered), first_line, and last_line. Do not wrap the JSON object in quotes.
        """
        response = firefall_service.completions(123, prompt, temperature=0.3, model_name="gpt-4-turbo")
        webpage_outline = response["generations"][0][0]["text"]

        #Remove ```json and  ``` from the string if the model is gpt-4-turbo
        if webpage_outline.startswith('```json') and webpage_outline.endswith('```'):
                    webpage_outline = webpage_outline[7:-3].strip()


        text_split = text.splitlines()
        text_joined = ' '.join([line.strip() for line in text_split])

        webpage_outline_json = json.loads(webpage_outline)

        # Get the first and last line of relevant text
        first_line = webpage_outline_json['first_line'].splitlines()
        last_line = webpage_outline_json['last_line'].splitlines()

        if len(first_line) > 1:
            first_line = ' '.join([line.strip() for line in first_line])
        else:
            first_line = first_line[0].strip()

        if len(last_line) > 1:
            last_line = ' '.join([line.strip() for line in last_line])
        else:
            last_line = last_line[0].strip()


        # Get the indices of the first and last line in the text
        first_line_index = text_joined.find(first_line)
        last_line_index = text_joined.rfind(last_line)

        if first_line_index == -1:
            first_line_index = 0

        if last_line_index == -1:
            last_line_index = len(text_joined)

        # Get the text between the first and last line
        relevant_text = text_joined[first_line_index:last_line_index]

        # Get the token encoding for the relevant text
        relevant_text_encoding = enc.encode(relevant_text)

        # Generate a prompt for the Firefall API
        session_id = 'scraping-20231214'
        prompt = f"""
        The following is the outline of a webpage. It includes a description of the main focus of the webpage and a list of all the corresponding sections and/or subsections. You are also provided with the webpage text.

        Your task is to format the webpage text so that it matches the outline while keeping the integrity of the text as it appears originally.
        Remove HTML tags if they are present in the text.
        ```outline
        {webpage_outline}
        ```

        ```webpage text
        {relevant_text}
        ```
        """
        response = firefall_service.completions(session_id, prompt, temperature=0.3, top_p=0.01, model_name="gpt-4-turbo", n=1)
        webpage_text = response["generations"][0][0]["text"]
        if webpage_text.startswith('```json') and webpage_text.endswith('```'):
                    webpage_text = webpage_text[7:-3].strip()

        #Remove ```json and  ``` from the string if the model is gpt-4-turbo
        webpage_text = webpage_text.replace('```formatted webpage text', '')
        webpage_text = webpage_text.replace('```', '')

        #add df.iloc[i]['top_keyword'] to the beginning of webpage_text string
        print(webpage_text)
        file_name = url.split('/')[-1]
        #and also before ?locale=de
        #file_name = file_name.split('?')[0]
        file_name = file_name.split('.')[0]
        file_name = f'data/{name_of_webpage}/scraped_content/{file_name}.md'
        utils.read_write_file(file_name,'w', webpage_text)

    except Exception as e:
        file_name = url.split('/')[-1]
        #file_name = file_name.split('?')[0]
        file_name = file_name.split('.')[0]
        file_name = f'data/{name_of_webpage}/scraped_content/{file_name}.md'
        utils.read_write_file(file_name,'w', text)

Processing row 0
https://www.lovesac.com/
Waiting for page to load...
200 {"conversation_identifier":null,"query_id":null,"llm_type":"azure_chat_openai","generations":[[{"text":"{\n  \"main_focus\": \"Sectionals & Bean Bags\",\n  \"sections\": 3,\n  \"first_line\": \"Sactionals are specially designed to adapt to your changing needs and evolving tastes.\",\n  \"last_line\": \"With our proprietary Durafoam fill, Sacs provide comfort for life.\"\n}","generation_info":{"finish_reason":"stop","content_filter_results":{"hate":{"filtered":false,"severity":"safe"},"self_harm":{"filtered":false,"severity":"safe"},"sexual":{"filtered":false,"severity":"safe"},"violence":{"filtered":false,"severity":"safe"}}},"type":"ChatGeneration","message":{"content":"{\n  \"main_focus\": \"Sectionals & Bean Bags\",\n  \"sections\": 3,\n  \"first_line\": \"Sactionals are specially designed to adapt to your changing needs and evolving tastes.\",\n  \"last_line\": \"With our proprietary Durafoam fill, Sacs provi